In [1]:
import os
import sys
import pandas as pd
import json
import networkx as nx
import plotly.io as pio
import plotly.graph_objects as go
from pathlib import Path

sys.path.append(os.path.abspath(os.path.join('..')))

from api_clients import UniProtAPIClient

# Set the renderer to 'browser' to force the graph to open in the browser
pio.renderers.default = 'browser'

In [2]:
# Directories and filenames
# Project directory --> Parent directory of the notebook
project_dir = Path(os.getcwd()).parent.parent
external_data_dir = os.path.join(project_dir, "external_data")
output_dir = os.path.join(project_dir, "output")
os.makedirs(output_dir, exist_ok=True)

drugbank_cleaned_csv_path = os.path.join(external_data_dir, "drugbank23/drugbank_clean.csv")

In [3]:
dbDF = pd.read_csv(drugbank_cleaned_csv_path)
print("Number of rows: ", len(dbDF))
print("Number of columns: ", len(dbDF.columns))

'''
    Data cleaning:
    1. Keep only small molecule drugs --> filter type column.
    2. Keep only approved drugs --> filter groups column (with contains).
    3. Remove nan values from targets column.
'''

dbDF = dbDF[dbDF['type'] == 'small molecule']
dbDF = dbDF[dbDF['groups'].str.contains('approved')]
dbDF = dbDF.dropna(subset=['targets']).reset_index(drop=True)

dbDF

Number of rows:  16762
Number of columns:  40


/var/folders/h9/k5st395556z0p6l0szwkp5pm0000gn/T/ipykernel_4551/2820500283.py:1: DtypeWarning:

Columns (29,30,31,32,36,37) have mixed types. Specify dtype option on import or set low_memory=False.



,type,created,updated,drugbank-id,name,description,cas-number,unii,state,groups,...,reactions,snp-effects,snp-adverse-drug-reactions,targets,carriers,transporters,fda-label,msds,average-mass,monoisotopic-mass
0,small molecule,2005-06-13,2023-01-03,DB00007,Leuprolide,Leuprolide is a synthetic 9-residue peptide an...,53714-56-0,EFY6W0M8TG,solid,approved investigational,...,NaN,NaN,NaN,BE0000203,NaN,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,NaN,1209.3983,1208.645462
1,small molecule,2005-06-13,2023-01-03,DB00014,Goserelin,"Goserelin is a synthetic hormone. In men, it s...",65807-02-5,0F65R8P09N,solid,approved,...,NaN,NaN,NaN,BE0000134 BE0000203,NaN,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,NaN,1269.4105,1268.641439
2,small molecule,2005-06-13,2023-01-03,DB00035,Desmopressin,"Desmopressin (dDAVP), a synthetic analogue of ...",16679-58-6,ENR1LLB0FP,solid,approved,...,NaN,NaN,NaN,BE0000773 BE0000165 BE0000293,NaN,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,//s3-us-west-2.amazonaws.com/drugbank/msds/DB0...,1069.2200,1068.426956
3,small molecule,2005-06-13,2023-01-03,DB00050,Cetrorelix,Cetrorelix is a man-made hormone that blocks t...,120287-85-6,OON1HFZ4BA,solid,approved investigational,...,NaN,NaN,NaN,BE0000134 BE0000203,NaN,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,//s3-us-west-2.amazonaws.com/drugbank/msds/DB0...,1431.0380,1429.669818
4,small molecule,2005-06-13,2023-01-03,DB00067,Vasopressin,Vasopressin (arginine-vasopressin or antidiure...,11000-17-2,Y87Y826H08,solid,approved,...,NaN,NaN,NaN,BE0000773 BE0000165 BE0000844 BE0000293,NaN,BE0001069,NaN,//s3-us-west-2.amazonaws.com/drugbank/msds/DB0...,2140.4600,2138.869562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,small molecule,2021-03-24,2022-12-06,DB16650,Deucravacitinib,Deucravacitinib is a novel oral selective tyro...,1609392-27-9,N0A21N6RAU,solid,approved investigational,...,BE0003541 P22309,NaN,NaN,BE0004146,NaN,BE0003659 BE0001032 BE0003648 BE0001067,NaN,NaN,425.4670,425.200317
1605,small molecule,2021-04-27,2023-01-03,DB16691,Nirmatrelvir,Nirmatrelvir (PF-07321332) is an orally bioava...,2628280-40-8,7R9A5P7H32,solid,approved investigational,...,NaN,NaN,NaN,BE0010016,NaN,BE0001032,NaN,NaN,499.5350,499.240639
1606,small molecule,2021-07-19,2022-12-01,DB16703,Belumosudil,Belumosudil is used in the treatment of chroni...,911417-87-3,834YJF89WO,solid,approved investigational,...,NaN,NaN,NaN,BE0001016 BE0004191,BE0000530 BE0000925,BE0001004 BE0001032 BE0001067,NaN,NaN,452.5180,452.196074
1607,small molecule,2022-03-24,2022-12-01,DB16778,Lutetium Lu-177 vipivotide tetraxetan,Lutetium Lu-177 vipivotide tetraxetan is a rad...,1703749-62-5,G6UF363ECX,liquid,approved,...,NaN,NaN,NaN,BE0008983,NaN,NaN,NaN,NaN,1216.0740,1215.422157


In [4]:
up_client = UniProtAPIClient()
drug_list = dbDF['drugbank-id'].values.tolist()

print("Number of drugs: ", len(drug_list))

target_dir = os.path.join(output_dir, "targets")
os.makedirs(target_dir, exist_ok=True)

'''for drug in drug_list:
    response = up_client.single_entity_search(from_db="DrugBank", to_db="UniProtKB", protein_id=drug)

    with open(os.path.join(target_dir, drug + ".json"), 'w') as outfile:
        json.dump(response, outfile)'''

dt_dict = {}
dt_idx = 0

for drug_file in os.listdir(target_dir):
    # Read json file
    with open(os.path.join(target_dir, drug_file), 'r') as infile:
        response = json.load(infile)

    if response:
        for target in response:
            dt_dict[dt_idx] = {
                "Drug":target["from"],
                "Protein":target["to"]["primaryAccession"],
            }

            dt_idx += 1

# dt_dict to df
dtDF = pd.DataFrame.from_dict(dt_dict, orient='index')
dtDF

Number of drugs:  1609


,Drug,Protein
0,DB00730,P00363
1,DB00730,P04798
2,DB00730,P05177
3,DB01121,P35498
4,DB11967,P02768
...,...,...
14027,DB15233,P10721
14028,DB15233,P11712
14029,DB15233,Q86VL8
14030,DB15233,Q96FL8


In [24]:
# Extract the unique targets
unique_targets = dtDF['Protein'].unique()
print("Number of unique targets: ", len(unique_targets))

# Extract the unique number of drugs mapped
unique_drugs = dtDF['Drug'].unique()
print("Number of unique drugs mapped successfully: ", len(unique_drugs))

Number of unique targets:  2310
Number of unique drugs mapped successfully:  1568


In [21]:
# Create the NetworkX graph
dtG = nx.from_pandas_edgelist(dtDF, source='Drug', target='Protein')

# Generate positions for each node using a spring layout
pos = nx.spring_layout(dtG)

# Extract node coordinates
x_nodes = [pos[node][0] for node in dtG.nodes()]
y_nodes = [pos[node][1] for node in dtG.nodes()]

# Extract edges and their positions
edge_x = []
edge_y = []
for edge in dtG.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])  # x coordinates for the edge
    edge_y.extend([y0, y1, None])  # y coordinates for the edge

# Identify node types: drugs or proteins
node_types = {node: ('Drug' if node in dtDF['Drug'].values else 'Protein') for node in dtG.nodes()}

# Color mapping for node types
node_color_map = {
    'Drug': 'red',
    'Protein': 'blue'
}

# Create a list of node colors based on their type
node_colors = [node_color_map[node_types[node]] for node in dtG.nodes()]

# Create edge traces for the graph
edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=1, color='gray'),
    hoverinfo='none',
    mode='lines'
)

# Create node traces with colors based on node type
node_trace = go.Scatter(
    x=x_nodes,
    y=y_nodes,
    mode='markers+text',
    text=[node for node in dtG.nodes()],  # Node labels
    hoverinfo='text',
    marker=dict(
        showscale=False,  # No color scale
        size=10,  # Node size
        color=node_colors,  # Color nodes by type (drug or protein)
        line_width=2
    )
)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title="Drug-Target Network Visualization",
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=40),
                    annotations=[dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper"
                    )],
                    xaxis=dict(showgrid=False, zeroline=False),
                    yaxis=dict(showgrid=False, zeroline=False)
                )
               )

# Show the graph
fig.show()


In [23]:
import pandas as pd
import networkx as nx

# Assuming dtG is your graph and dtDF contains your drug and protein data.

# 1. Number of nodes
num_nodes = dtG.number_of_nodes()

# 2. Number of edges
num_edges = dtG.number_of_edges()

# 3. Degree of each node
degree_dict = dict(dtG.degree())
degree_df = pd.DataFrame.from_dict(degree_dict, orient='index', columns=['degree'])
degree_df['Node'] = degree_df.index
degree_df = degree_df.reset_index(drop=True)

# Add a type column: Drug or Protein
degree_df['Type'] = degree_df['Node'].apply(lambda x: 'Drug' if x in dtDF['Drug'].values else 'Protein')

# Drugs degree
drug_degree_df = degree_df[degree_df['Type'] == 'Drug'].sort_values(by='degree', ascending=False).reset_index(drop=True)
# Proteins degree
proteins_degree_df = degree_df[degree_df['Type'] == 'Protein'].sort_values(by='degree', ascending=False).reset_index(drop=True)

# 4. Connected components
connected_components = list(nx.connected_components(dtG))
num_connected_components = len(connected_components)
# Sort the connected components by size in descending order
connected_components = sorted(connected_components, key=len, reverse=True)

# 5. Graph density
graph_density = nx.density(dtG)

# Centrality Metrics
centrality_metrics = {}

# 6. Degree Centrality
degree_centrality = nx.degree_centrality(dtG)
centrality_metrics['Degree Centrality'] = pd.DataFrame.from_dict(degree_centrality, orient='index', columns=['Degree Centrality'])

# 7. Closeness Centrality
closeness_centrality = nx.closeness_centrality(dtG)
centrality_metrics['Closeness Centrality'] = pd.DataFrame.from_dict(closeness_centrality, orient='index', columns=['Closeness Centrality'])

# 8. Betweenness Centrality
betweenness_centrality = nx.betweenness_centrality(dtG)
centrality_metrics['Betweenness Centrality'] = pd.DataFrame.from_dict(betweenness_centrality, orient='index', columns=['Betweenness Centrality'])

# 9. Eigenvector Centrality
eigenvector_centrality = nx.eigenvector_centrality(dtG, max_iter=1000)
centrality_metrics['Eigenvector Centrality'] = pd.DataFrame.from_dict(eigenvector_centrality, orient='index', columns=['Eigenvector Centrality'])

# 10. Clustering Coefficient
clustering_coefficient = nx.clustering(dtG)
centrality_metrics['Clustering Coefficient'] = pd.DataFrame.from_dict(clustering_coefficient, orient='index', columns=['Clustering Coefficient'])

# 11. Average Path Length
average_path_lengths = []
for component in connected_components:
    subgraph = dtG.subgraph(component)
    if len(subgraph) > 1:  # Only compute if the component has more than one node
        average_path_lengths.append(nx.average_shortest_path_length(subgraph))
    else:
        average_path_lengths.append(float('inf'))  # Not defined for single-node components

# Print results
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
print(f"Graph density: {graph_density:.4f}")
print(f"Number of connected components: {num_connected_components}")

# Optional: print the size of each connected component
for i, component in enumerate(connected_components):
    print(f"Size of component {i + 1}: {len(component)}")
    print(f"Average Path Length for component {i + 1}: {average_path_lengths[i]:.4f}")

# Display centrality metrics
for metric_name, df in centrality_metrics.items():
    df['Node'] = df.index
    df = df.reset_index(drop=True)
    print(f"\n{metric_name}:\n{df}")


Number of nodes: 3878
Number of edges: 14032
Graph density: 0.0019
Number of connected components: 43
Size of component 1: 3763
Average Path Length for component 1: 4.3720
Size of component 2: 10
Average Path Length for component 2: 1.9556
Size of component 3: 6
Average Path Length for component 3: 1.6667
Size of component 4: 5
Average Path Length for component 4: 1.6000
Size of component 5: 4
Average Path Length for component 5: 1.6667
Size of component 6: 4
Average Path Length for component 6: 1.5000
Size of component 7: 3
Average Path Length for component 7: 1.3333
Size of component 8: 3
Average Path Length for component 8: 1.3333
Size of component 9: 3
Average Path Length for component 9: 1.3333
Size of component 10: 3
Average Path Length for component 10: 1.3333
Size of component 11: 3
Average Path Length for component 11: 1.3333
Size of component 12: 3
Average Path Length for component 12: 1.3333
Size of component 13: 3
Average Path Length for component 13: 1.3333
Size of compone